# **Projet méthodologie de recherche : Non-linear Attributed Graph Clustering by Symmetric NMF with PU Learning**

Réalisé par : Oubenyahya Sanae & Hilal Chaimae & Ettali Mouad & Ouazzani Chahdi Kenza

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import random

def VU_init(A,k1,k2,flag,data):
    n = A.shape[0]
    m = A.shape[1]
    if flag == 0:
        U = np.random.random((n,k1))
        V = np.random.random((m,k2))
    else:
        infile = 'initialize/'+data+'_U_'+str(k1)+'.csv'
        U = []
        with open(infile) as f:
            while True:
                row = []
                line = f.readline()
                if line == '':
                    break
                line = line.rstrip('\n')
                tmp = line.split(',')
                for i in tmp:
                    row.append(float(i))
                U.append(row)
        infile = 'initialize/'+data+'_V_'+str(k2)+'.csv'
        V = []
        with open(infile) as f:
            while True:
                row = []
                line = f.readline()
                if line == '':
                    break
                line = line.rstrip('\n')
                tmp = line.split(',')
                for i in tmp:
                    row.append(float(i))
                V.append(row)
        U = np.array(U)
        V = np.array(V).transpose()
    return U,V

In [ ]:
"""
NJNMF with PU-learning is described in this program.
We recommend using kmeans initialization (init=1).
In that case, you run kmeans_init.py in advance.
"""

import time
import sys
import scipy.stats
import numpy as np
import scipy as sp

# import networkx as nx
# from matplotlib import pyplot
# max_iter=100 #number of iterations
b=0.0   #bias for sigmoid function

class NAGC:
    def __init__(self,k1,k2,lam,S,W,X,data,init=1,rho=0.5,max_iter=100):
        node_size = S.shape[0]
        att_size = X.shape[1]
        #H is a matrix between topic1 and topic2
        H = np.random.random((k1,k2))
        #create initialized matrices
        U,V = VU_init.VU_init(X,k1,k2,init,data)
        # a = (1.0+rho)/2
        # W = np.abs(S_ori-1)
        W_ = 1-W
        # general params
        self.S = S
        self.W = W
        self.W_ = W_
        self.X = X
        self.max_iter = max_iter
        self.U = U
        self.H = H
        self.V = V
        self.lam = lam
        self.rho = rho
    def fit_predict(self):
        def update_U(S,W,W_,X,lam,U,H,V,rho):
            fUH = sigmoid(U.dot(H))
            fdUH = dif_sig(U.dot(H))
            UUT = U.dot(U.transpose())
            U = U*((rho*2.0)*S.dot(U)+(lam*X.dot(V) * fdUH).dot(H.transpose()))/((2.0*rho)*(UUT*W).dot(U)+(2.0*(1.0-rho))*(UUT*W_).dot(U)+(lam*fUH.dot(V.transpose().dot(V)) * fdUH).dot(H.transpose()))
            #V = V*((a*2.0)*S.dot(V)+(lam*A.dot(U) * fdVT).dot(T.transpose()))/(((2.0*a)*(VVT*S_ori)+(2.0*(1.0-a))*(VVT*S_)).dot(V)+(lam*fVT.dot(U.transpose().dot(U)) * fdVT).dot(T.transpose()))
            return U
        def update_U_woPU(S,X,lam,U,H,V):
            fUH = sigmoid(U.dot(H))
            fdUH = dif_sig(U.dot(H))
            U = U*(2.0*S.dot(U)+(lam*X.dot(V) * fdUH).dot(H.transpose()))/(2*U.dot(U.transpose().dot(U))+(lam*fUH.dot(V.transpose().dot(V)) * fdUH).dot(H.transpose()))
            return U
        def update_V(S,X,U,H,V):
            # fVT = sigmoid(V.dot(T))
            return V*(X.transpose().dot(sigmoid(U.dot(H))))/(V.dot(sigmoid(U.dot(H).transpose()).dot(sigmoid(U.dot(H)))))
        def update_H(S,X,U,H,V):
            fUH = sigmoid(U.dot(H))
            fdUH = dif_sig(U.dot(H))
            H = H*(U.transpose().dot(fdUH*(X.dot(V))))/(U.transpose().dot((fdUH*fUH).dot(V.transpose().dot(V))))
            return H
        def removing_nan(mat):
            nan_list = np.argwhere(np.isnan(mat))
            for i in nan_list:
                mat[i[0],i[1]]=sys.float_info.epsilon
            return mat
        def sigmoid(x):
            return 1.0 / (1.0 + np.exp(-(x-b)))
        def dif_sig(x):
            return 1.0 * np.exp(-(x-b)) / pow(1.0+np.exp(-(x-b)),2)

        start = time.time() #memo start time
        #learning step
        count = 0
        while 1:
            count += 1
            # print loss_function(S,V,U,Z,A,T,lam)
            if self.rho == 0.5:
                self.U = removing_nan(update_U_woPU(self.S,self.X,self.lam,self.U,self.H,self.V))
            else:
                self.U = removing_nan(update_U(self.S,self.W,self.W_,self.X,self.lam,self.U,self.H,self.V,self.rho))
            self.V = removing_nan(update_V(self.S,self.X,self.U,self.H,self.V))
            self.H = removing_nan(update_H(self.S,self.X,self.U,self.H,self.V))
            if count>=self.max_iter:
                break
        elapsed_time = time.time() - start  #measure elapsed time
        print (("optimizing_time:{0}".format(elapsed_time)) + "[sec]")
        return self.U

In [ ]:
#This program build adjacency matrix and attribute matrix from graph data.
#This program can deal with two format(.mat and .cite&.content)
#Outputs are blow
'''
S : adjacency matrix with prepocess
S_ori : original adjacency matrix
A : attribute matrix with preprocess
clus : true cluster of nodes
flag : with ground truth or without
A_ori : original attribute matrix
'''

import numpy as np
import glob
import scipy.io

def build_graph(path): # switch for two form of file
    if "mat" in path:
        print ("mat")
        S,S_ori,A,clus,A_ori = for_mat(path)
        flag=1
        if clus==[]:
            flag=0
        return S,S_ori,A,clus,flag,A_ori
    else:
        print ("cite")
        S,S_ori,A,clus,A_ori = for_cites_contents(path)
        return S,S_ori,A,clus,1,A_ori

# def for_mat(path):
#     matdata = scipy.io.loadmat(path)
#     a = matdata["A"]
#     f = matdata["F"]
#     node_size = a.shape[0]
#     att_size = f.shape[1]
#     # S = lil_matrix((node_size,node_size))
#     S = np.zeros((node_size,node_size))
#     A = np.zeros((node_size,att_size))
#     #fill the adjacency matrix and attribute matrix
#     nonzeros = a.nonzero()
#     print ("no.nodes: " + str(node_size))
#     print ("no.attributes: " + str(att_size))
#     edgecount=0
#     for i in range(len(nonzeros[0])):
#         S[nonzeros[0][i],nonzeros[1][i]] = 1
#         if nonzeros[0][i]<=nonzeros[1][i]:
#             edgecount+=1
#     print ("no.edges: " + str(edgecount))
#     # S=S.tocsr()
#     nonzeros = f.nonzero()
#     for i in range(len(nonzeros[0])):
#         A[nonzeros[0][i]][nonzeros[1][i]] = f[nonzeros[0][i],nonzeros[1][i]]
#     # print A
#     # reg=1
#     # A_copy = copy.deepcopy(A)
#     # if reg == 1:
#     #     for i in range(A_copy.shape[1]):
#     #         max_att = max(A_copy[:,i])
#     #         if max_att != 0:
#     #             A_copy[:,i] = A_copy[:,i]/max_att*100
#     S_pre,A_pre=preprocess(S,A)
#     return S_pre,S,A_pre,[],A #scaling S and A

def for_mat(path):
    mat_contents = scipy.io.loadmat(path)
    G = mat_contents["Network"]
    X = mat_contents["Attributes"]
    Label = list(map(int,mat_contents["Label"]))
    node_size = G.shape[0]
    att_size = X.shape[1]
    # S = lil_matrix((node_size,node_size))
    # S = np.zeros((node_size,node_size))
    # A = np.zeros((node_size,att_size))
    S = np.zeros((node_size,node_size))
    A = X.toarray()
    #fill the adjacency matrix and attribute matrix
    nonzeros = G.nonzero()
    print ("no.nodes: " + str(node_size))
    print ("no.attributes: " + str(att_size))
    edgecount=0
    for i in range(len(nonzeros[0])):
        S[nonzeros[0][i],nonzeros[1][i]] = 1
        S[nonzeros[1][i],nonzeros[0][i]] = 1
    # erase diagonal element
    diag = 0
    for i in range(node_size):
        diag += S[i,i]
    nonzeros = S.nonzero()
    edge_count = int((len(nonzeros[0])+diag)/2)
    print ("number of edges : " + str(edge_count))

    S_pre,A_pre=preprocess(S,A)
    return S_pre,S,A_pre,Label,A #scaling S and A


def for_cites_contents(path):
    node={}
    counter=0
    att_list=[]
    clus=[]
############ download atttributes #################
    infiles = glob.glob(path+'/*.content')
    for infile in infiles:
        with open(infile) as f:
            while True:
                line = f.readline()
                if line == '':
                    break
                tmp = line.split("\t")
                node[tmp[0]]=counter
                counter+=1
                att_list.append((tmp[1:-1]))
                clus.append(tmp[-1].replace("\n",""))
    print ("number of nodes : " + str(len(node)))
    print ("number of attributes : " + str(len(att_list[0])))
    # print (clus)
############ download edges #################
    edges=[]
    infiles = glob.glob(path+'/*.cites')
    for infile in infiles:
        with open(infile) as f:
            while True:
                line = f.readline()
                if line == '':
                    break
                line = line.replace("\n","")
                tmp = line.split()
                if tmp[0] in node and tmp[1] in node:
                    ind0 = node[tmp[0]]
                    ind1 = node[tmp[1]]
                    edges.append((ind0,ind1))
    node_size = len(node)
    att_size = len(att_list[0])
    # S = lil_matrix((node_size,node_size))
    S = np.zeros((node_size,node_size))
    # A = lil_matrix((node_size,att_size))
    A = np.zeros((node_size,att_size))
    for i in range(len(edges)):
        S[edges[i][0],edges[i][1]] = 1
        S[edges[i][1],edges[i][0]] = 1
    # erase diagonal element
    diag = 0
    for i in range(node_size):
        diag += S[i,i]
    nonzeros = S.nonzero()
    edge_count = int((len(nonzeros[0])+diag)/2)
    print ("number of edges : " + str(edge_count))
    # S=S.tocsr()
    for i in range(len(att_list)):
        for j in range(len(att_list[0])):
            A[i,j] = float(att_list[i][j])
    # A=A.tocsr()

    S_pre,A_pre=preprocess(S,A)
    return S_pre,S,A_pre,clus,A #scaling S and A

def preprocess(S,A):
    # initialization in the paper(JWNMF)
    # S = S / S.sum()
    # A = A / A.sum()

    # initialization based on size of S
    # A = A * S.sum() / A.sum()
    S = S * A.sum() / S.sum()
    return S,A

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#this program include calculater of modularity, entropy, NMI and ARI
import sys
import scipy.stats
import numpy as np
import scipy as sp
import scipy.io
from scipy.sparse import lil_matrix, csr_matrix
import math
from sklearn.metrics.cluster import adjusted_rand_score

def evaluate(mod,ent,spl,nmi,ari,true_clus,clus,pred,S,A,ent_flag,with_gt):
    k = len(clus)
    mod.append(cal_modularity(clus,S,k))
    if ent_flag == 0:
        ent.append(cal_entropy(clus,A,k))
    elif ent_flag == 1:
        ent.append(cal_dif_entropy(clus,A_copy,k))
        spl.append(split_entropy(clus,A_ori,k))
    if with_gt >= 3:
        nmi.append(cal_nmi(true_clus,clus))
        ari.append(ARI(true_clus,pred))
    return mod,ent,spl,nmi,ari
#this function calculates modularity from clustering result
def cal_modularity(clus,S,k):
    a = [0]*k
    e = np.zeros((k,k))
    for i in range(k):
        for j in range(k):
            for l in clus[i]:
                for m in clus[j]:
                    e[i][j]+=float(S[l,m])
    # regularize e on the sum of S
    e = e / S.sum()
    for i in range(k):
        a[i] = sum(e[i][:])
    Q=0
    for i in range(k):
        Q+=e[i][i]-a[i]*a[i]
    return Q

def cal_entropy(clus,A,k):
    E=0
    for t in range(A.shape[1]):
        for j in range(k):
            if len(clus[j])==0:
                continue
            att=0
            for n in clus[j]:
                if A[n,t]==0:
                    att+=1
            pk=[1.0*att/len(clus[j]),1.0-1.0*att/len(clus[j])]
            # print pk
            E += len(clus[j]) * scipy.stats.entropy(pk)
    return E/A.shape[0]/A.shape[1]

def cal_nmi(true_clus, clus):
    t_label_list = list(set(true_clus))
    n_h=[]
    for i in range(len(t_label_list)):
        n_h.append(true_clus.count(i))
    t_clus_ind = [[] for i in range(len(n_h))]
    for i in range(len(true_clus)):
        t_clus_ind[true_clus[i]].append(i)
    n_l = []
    for i in range(len(clus)):
        n_l.append(len(clus[i]))

########### print No. labels ###############
    print("estimate_No.label : [ ",end="")
    for i in range(len(n_l)):
        print(str(n_l[i])+" ",end="")
    print(']')
#############################################
    n=sum(n_l)
    nmi = 0
    for h in range(len(n_h)):
        for l in range(len(n_l)):
            n_hl = len(list(set(t_clus_ind[h]) & set(clus[l])))
            if n_hl != 0 and n_h[h] != 0 and n_l[l] != 0:
                nmi += n_hl * math.log(1.0*n*n_hl/n_h[h]/n_l[l],2)
    deno_h = 0
    for h in range(len(n_h)):
        if n_h[h] != 0:
            deno_h += n_h[h] * math.log(1.0*n_h[h]/n,2)
    deno_l = 0
    for l in range(len(n_l)):
        if n_l[l] != 0:
            deno_l += n_l[l] * math.log(1.0*n_l[l]/n,2)
    if deno_h*deno_l != 0.0:
        nmi = nmi/math.sqrt(deno_h*deno_l)
    else:
        nmi = 0.0
    return nmi

def ARI(true_clus,clus):
    return adjusted_rand_score(true_clus,clus)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from sklearn.cluster import KMeans
import argparse

import numpy as np
import csv

def clustering(A,k):
    kmeans_clus=[]
    for i in range(k):
        kmeans_clus.append([])
############# regularization for input attributes
    for i in range(A.shape[1]):
        max_att = max(A[:,i])
        if max_att != 0:
            A[:,i] = A[:,i]/max_att
############ Learing step of kmeans
    kmeans = KMeans(n_clusters=k).fit(A)
    pred = kmeans.labels_
    centroids = kmeans.cluster_centers_
    for i in range(len(pred)):
        kmeans_clus[pred[i]].append(i)
    return pred, centroids, kmeans_clus
def initialize_U(A, centroids):
    U = np.zeros((len(A),len(centroids)))
    for i in range(U.shape[0]):
        dis_list = []
        for j in range(U.shape[1]):
            dis_list.append(np.linalg.norm(A[i]-centroids[j]))
        for j in range(U.shape[1]):
            U[i,j]= (sum(dis_list)-dis_list[j]) / sum(dis_list)
        U[i,:] = U[i,:] / sum(U[i,:])
    return U

def init_kmeans(k,data):
    print(data)

    path = "data/"+data
    S, S_ori, A, true_clus, flag, A_ori = build_graph(path)
    clus_list = list(set(true_clus))
    print(clus_list)
    clus_dic = {}
    for i in range(len(clus_list)):
        clus_dic[clus_list[i]]=i
    for i in range(len(true_clus)):
        true_clus[i] = clus_dic[true_clus[i]]

    pred_l=[];cent_l=[];km_l=[]
    mod=[];ent=[];nmi=[];ari=[]
    for j in range(5):
        pred, centroids, kmeans_clus = clustering(A_ori,k)
        pred_l.append(pred)
        cent_l.append(centroids)
        km_l.append(kmeans_clus)
        ari.append(evaluate.ARI(true_clus,pred))
    ind=ari.index(sorted(ari)[2])
    pred=pred_l[ind]
    centroids=cent_l[ind]
    kmeans_clus=km_l[ind]
    U = initialize_U(A_ori, centroids)
    f = open('initialize/'+data+'_U_'+str(k)+'.csv','w')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(U)
    f.close()
    f = open('initialize/'+data+'_V_'+str(k)+'.csv','w')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(centroids)
    f.close()

#**NAGC EXEMPLE**

In [ ]:
import NAGC_class
import build_graph
import evaluate
import init_kmeans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data="WebKB_univ"
#data="citeseer"
# data = "polblog"
# data = "cora"
data_path = "/content/drive/MyDrive/data/data/"+data
S, W, X, true_clus, flag, A_ori = build_graph(data_path)


cite
number of nodes : 877
number of attributes : 1703
number of edges : 1480


In [ ]:
k1=4 # number of clusters you want to extract
k2=20 # number of topic2 

##**Kmeans**

In [ ]:
init_kmeans(k1,data)

In [ ]:
init_kmeans(k2,data)

##**Modifier les parametres et executer le model**

In [ ]:
lam=0.0001 # balancing parameter between the topology and attributes
rho=0.55
iteration=100 # number of iterations
init=1 # 0: random initialization, 1: kmeans initialization

NAGC = NAGC_class.NAGC(k1,k2,lam,S,W,X,data,init,rho)
U = NAGC.fit_predict()

##**Evaluation des résultats de clustering**

In [ ]:
k = k1
clus=[]
for i in range(U.shape[1]):
    clus.append([])
pred = U.argmax(1)
for i in range(U.shape[0]):
    clus[pred[i]].append(i)
print("modularity: " + str(evaluate.cal_modularity(clus,S,k)))
print("entropy: " + str(evaluate.cal_entropy(clus,X,k)))
if true_clus != []:
    print("ARI: " + str(evaluate.ARI(true_clus,pred)))